In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import random

In [2]:
from numba import jit
import numpy as np

In [3]:
@jit(nopython=True, cache=True, fastmath=True)
def make_sphere_array(radius):
    arr = np.full((2*radius+1,2*radius+1,2*radius+1), False)
    x0, y0, z0 = int(arr.shape[0]/2),int(arr.shape[1]/2),int(arr.shape[2]/2)
    for x in range(x0-radius, x0+radius+1):
        for y in range(y0-radius, y0+radius+1):
                for z in range(z0-radius, z0+radius+1):
                    deb = np.linalg.norm(np.array([x0-x, y0-y, z0-z], dtype=np.float32))
                    arr[x,y,z] = True if deb <= radius else False
    return arr

def search_light_analysis(data, radius, search_area=None, post_func=None):
    search_area = data if search_area is None else search_area 
    post_func = (lambda x:x) if post_func is None else post_func
    sphere_mask = make_sphere_array(radius)
    for x0 in range(radius, data.shape[0]-radius):
        for y0 in range(radius, data.shape[1]-radius):
            for z0 in range(radius, data.shape[2]-radius):
                if search_area[x0,y0,z0]:
                    target_area = data[x0-radius:x0+radius+1,y0-radius:y0+radius+1,z0-radius:z0+radius+1].copy()
                    available_mask = (target_area.sum(axis=3)!=0) & sphere_mask
                    if np.any(available_mask):
                        values = target_area[available_mask]
                        result = post_func(values)
                        yield result, (x0,y0,z0)

In [4]:
beta = np.load('Sample_fMRI_data.npz')['beta']

# Define mask
mask = np.ones((beta.shape[:3]))
tmp = np.sum((beta!=0),axis=3)
mask[(tmp<16)] = 0

# Searchlight
brain_inform_list = [inform for inform in search_light_analysis(beta,radius=3,search_area=mask)]
nvv = len(brain_inform_list) # number of valid voxels
brain_rdm = np.zeros((nvv,16,16))
cnt_info = np.zeros((nvv),dtype=int)
for vi in range(nvv): 
    tp = brain_inform_list[vi][0] # nvv,192
    tp2 = 1-np.corrcoef(tp, rowvar=False)
    brain_rdm[vi] = tp2
    cnt_info[vi] = tp.shape[0]
    


In [37]:
brain_inform_list[6][0].shape

(55, 16)

In [5]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
# Create RDM codes for each condition
# Digit recognition code
dgt_cd = np.zeros((16,16))
dgt_cd[:10,:10] = 1

# Object recognition code
obj_cd = np.zeros((16,16))
obj_cd[10:,10:] = 1

# Digit vs. Object code
vs_cd = np.zeros((16,16))
vs_cd[:10,10:] = 1
vs_cd[10:,:10] = 1

# Magnitude code
mag_cd = np.zeros((16,16))
mag_cd[7:10,1:4] = 1

# Animacy code
ani_cd = np.zeros((16,16))
ani_cd[13:,10:13] = 1

cd_name_list = ['Digit', 'Object', 'Versus', 'Magnitude', 'Animacy']    
cds_list = [dgt_cd,obj_cd,vs_cd,mag_cd,ani_cd]
num_cds = len(cds_list)

In [7]:
def rdm_reord(org):
    new = org.copy()
    new[10:15,10:15] = org[11:,11:]
    new[15,15] = org[10,10]
    new[-1,10:15] = org[10,11:]
    new[10:15,-1] = org[11:,10]
    return new

il16 = np.tril_indices(16,-1)
def rsa(rdm_,code):
    rho, p = stats.spearmanr(code[il16],rdm_[il16])
    return rho, p 

In [8]:
nvv = brain_rdm.shape[0]

# RSA
rsa_rslt = np.zeros((num_cds,64,76,64))
rsa_rslt_pval = np.zeros((num_cds,64,76,64))
for cdi,code in enumerate(cds_list):
    vol_rho = np.zeros((64,76,64))
    vol_pval = np.zeros((64,76,64))
    for vi in range(nvv):
        idx = brain_inform_list[vi][1]
        rdm_ = rdm_reord(brain_rdm[vi])
        vol_rho[idx], vol_pval[idx] = rsa(rdm_,code)
    rsa_rslt[cdi], rsa_rslt_pval[cdi] = vol_rho, vol_pval

In [9]:
brain_rdm.shape

(33996, 16, 16)

ROI mao associated with each concept

In [10]:
p_threshold = 0.05
rsa_rslt[(rsa_rslt_pval > p_threshold)] = 0 # uncorrected p-values used in this sample code
rsa_rslt[(rsa_rslt < 0)] = 0 # Positive voxels

In [11]:
import nibabel as nib

affine = np.array([[   3. ,   -0. ,   -0. ,  -94.5],
       [  -0. ,    3. ,   -0. , -130.5],
       [   0. ,    0. ,    3. ,  -76.5],
       [   0. ,    0. ,    0. ,    1. ]])

def save_nifti(vol, savepath, zero_to_nan=True):
    if zero_to_nan:
        vol[(vol==0)] = np.nan
    img1 = nib.Nifti1Image(vol,affine=affine)
    nib.save(img1,savepath)

for cdi in range(num_cds):    
    save_nifti(rsa_rslt[cdi], cd_name_list[cdi]+'.nii')

In [15]:
num_cds = 5
cd_name_list = ['Digit', 'Object', 'Versus', 'Magnitude', 'Animacy']    
roi_masks = np.zeros((num_cds,64,76,64))
for cdi in range(num_cds):
    tmp = np.nan_to_num(nib.load(cd_name_list[cdi]+'.nii').get_fdata())
    roi_masks[cdi] = (tmp!=0)

In [19]:
nvv = brain_rdm.shape[0]


for cdi in range(num_cds):
    stim_type = cd_name_list[cdi]
    roi_mask = roi_masks[cdi]
    for vi in range(nvv):
        idx = brain_inform_list[vi][1]
        # If the index is not at the ROI, continue.
        if roi_mask[idx] != 1:
            continue
        roirdm = brain_rdm[vi]


Calculate the RSA results for each concept and ROI
